# voterecord.py

- Record 클래스 보유.
    - 필요 생성자 : csvline
        - csvline : 주어진 csv 파일의 행.

In [ ]:
# 해당 record 줄에 대한 분석.
class Record:

    types = ['republican', 'democrat']
    values = ['y', 'n', '?']
    numValues = 16

    # 정당과 투표 결과값 분리.
    def __init__(self, csvline):
        self.party = csvline[0]    # csvline의 첫 번째 요소는 정당. 따로 빼두기.
        self.feature = []    # 법안에 대한 특정 정치인의 투표 결과를 저장할 리스트.
        for itr in range(1, len(csvline)):    # 해당 csvline에 있는 투표 결과 모두(정당 제외) 위 리스트에 저장.
            self.feature.append(csvline[itr])

    def __str__(self):    # 인스턴스 자체를 출력 할 때의 형식을 지정해주는 함수.
                          # 후에 Record instance을 생성하고 생성된 instance를 출력할 때 사용됨.
        ret = 'Classification : '+self.party+', Features : '+str(self.feature)
        return ret

# decisiontreenode.py

In [ ]:
import csv
import math
from voterecord import Record    # 위에 작성했던 Record 클래스 사용.

class Node:

## def \__init__( ):

In [3]:
    def __init__(self, records):
        self.blnSplit = False    # boolean split. split을 했는지 확인해주는 불린.
                                 # splitNode() 함수 사용하면 True됨.
        self.children = {}    # root의 child 초기화인듯.
        self.records = records    # Record() 함수로 받아온 레코드.
        self.decisionAttribute = -1    # 결정에 영향을 주는 Attribute가 몇갠지. 초기값은 -1.
        self.stat = {}    # 공화당/민주당 몇명인지 보여주는 딕셔너리 생성.

        # 각각 정당인이 몇명인지. 
        # result : {'republican': 168, 'democrat': 267}
        for type in Record.types:    # 공화당, 민주당 둘 중 하나
            self.stat[type] = 0    # 각각 정당의 type란에 0 저장.
            for record in records:
                if record.party == type:    # 해당 record 라인의 정당이 type과 일치한다면
                    self.stat[type] = self.stat[type] + 1    # 해당 값에 1 더하기.   

{'Repub': 0}

## def \__str__( ):

In [ ]:
    def __str__(self):    # Node로 만든 instance 실행될 때 출력.
        if self.blnSplit == True:    # splitNode() 함수를 사용하면 True가 된다.
            ret = 'Feature '+str(self.decisionAttribute)+ '('+str(self.stat)+')' + '\n'
            for key in self.children.keys():    # information gains를 나타내는 keys()
                ret = ret + '---- key : '+str(key)+ ' '+str(self.children[key])
        else:    # 영향 미치는 factor가 없다면
            ret = str(self.stat) + '\n'
        return ret 

## def splitNode( ):

In [ ]:
    def splitNode(self):
        self.blnSplit = True
        gains = self.calculateInformationGainPerFeatures()    # 정보이득 저장된 값.
        idxMaxGainFeature = -1
        maxGain = -999999999999999999.0
        for itr in range(len(gains)):    # 16번 for loop.
            if maxGain < gains[itr]:
                maxGain = gains[itr]    # 가장 의사결정에 도움이 되는 infoGain.
                idxMaxGainFeature = itr    # 해당 infoGain의 인덱스.
                
        for value in Record.values:    # y, n, ? 세개.
            childRecords = []
            for record in self.records:   # record 마다
                if record.feature[idxMaxGainFeature] == value:   # 해당 record의 해당 인덱스(여기선 3)에 그 value가 있다면:
                    childRecords.append(record)    # 해당 record를 childRecords 리스트에 저장.
            self.children[value] = Node(childRecords)
        self.decisionAttribute = idxMaxGainFeature
        return self.children

## def calculateInformationGainPerFeatures( ):

In [ ]:
    # Feature(투표 결과. 16개)마다 정보이득(information gain) 계산.
    # gains value가 클수록 해당 리뷰가 의사결정에 도움이 된다는 뜻.
    def calculateInformationGainPerFeatures(self):
        gains = []    # 계산한 정보이득 값들 저장할 곳.
        entropyClass = self.calculateClassEntropy()    # 0.9623080486960709. 모든 투표 결과에 대한 entropy.
        for itr in range(Record.numValues):    # 0부터 15 총 16개.
            entropyConditional = self.calculateConditionalEntropy(itr)    
            gains.append(entropyClass-entropyConditional)
        return gains

## def calculateClassEntropy( ):

In [ ]:
    # 전체 결과에 대한 entropy. 공화/민주당원이 몇명인지에 따른 entropy.
    def calculateClassEntropy(self):
        entropy = 0.0    # 초기 엔트로피 값.
        for type in Record.types:    # 공화/민주
            cnt = 0.0
            for record in self.records:    # 밑에서 저장한 records값.
                if record.party == type:    #
                    cnt = cnt + 1.0
            size = float(len(self.records))    # 전체 레코드 수.
            prob = float(cnt / size)    
            entropy = entropy - prob * math.log(prob, 2)
        return entropy

## calculateConditionalEntropy( ):

In [ ]:
    # 특정 투표의 결과에 대한 엔트로피. 결과가 응집력이 높을수록 엔트로피가 낮다.
    def calculateConditionalEntropy(self, idxFeature):
        entropy = 0.0
        for value in Record.values:    # y, n, ? 세개.
            for type in Record.types:    # 공화/민주
                cntFeature = 0.0    # 해당 법안에 대한 투표 결과값
                cntFeatureAndClass = 0.0
                for record in self.records:
                    if record.feature[idxFeature] == value:
                        cntFeature = cntFeature + 1
                        if record.party == type:
                            cntFeatureAndClass = cntFeatureAndClass + 1.0
                size = float(len(self.records))
                probFeature = cntFeature / size + 0.000001
                probFeatureAndClass = cntFeatureAndClass / size + 0.000001
                entropy = entropy + probFeatureAndClass * math.log(probFeature/probFeatureAndClass, 2)
        return entropy

## if \__name\__ == "\_main\_":

In [ ]:
# 이 파일을 실행했을 때 이하의 구문들이 출력된다. 오로지 import만 했을 경우에는 실행되지 않는다.
# 아마 이번 실습에는 필요없고 다음 실습에서 사용될때를 위한 것 같다.
# 자세한 설명은 https://wikidocs.net/29 여기서.
if __name__ == "__main__":
    csvfile = open('house-votes-84.csv','rt')    # rt: text모드를 read.
    reader = csv.reader(csvfile,delimiter=',')
    records = []

    for row in reader:
        record = Record(row)    # 각 행마다 정당/결과 분리
        print (record)    # __str__ 함수에 의해 출력.
        records.append(record)

    node = Node(records)
    print(node)
    node.splitNode()
    print(node)

# 출력 결과

In [ ]:
print(record) 출력 결과
'''
Classficiation : republican, Features : ['n', 'y', 'n', 'y', 'y', 'y', 'n', 'n', 'n', 'y', '?', 'y', 'y', 'y', 'n', 'y']
Classficiation : republican, Features : ['n', 'y', 'n', 'y', 'y', 'y', 'n', 'n', 'n', 'n', 'n', 'y', 'y', 'y', 'n', '?']
.
.
.
Classficiation : republican, Features : ['n', 'n', 'n', 'y', 'y', 'y', '?', '?', '?', '?', 'n', 'y', 'y', 'y', 'n', 'y']
Classficiation : republican, Features : ['n', 'y', 'n', 'y', 'y', 'y', 'n', 'n', 'n', 'y', 'n', 'y', 'y', 'y', '?', 'n']
'''

print(node) 출력 결과
'''
{'republican': 168, 'democrat': 267}
'''

node.split() 후 print(node) 출력 결과
'''
Feature 3({'republican': 168, 'democrat': 267})
---- key : y {'republican': 163, 'democrat': 14}
---- key : n {'republican': 2, 'democrat': 245}
---- key : ? {'republican': 3, 'democrat': 8}
'''

# 전체 코드

In [ ]:
import csv
import math
from voterecord import Record

class Node:
    
    def __init__(self, records):
        self.blnSplit = False
        self.children = {}
        self.records = records
        self.decisionAttribute = -1
        self.stat = {}
        
        for type in Record.types:
            self.stat[type] = 0
            for record in records:
                if record.party == type:
                    self.stat[type] = self.stat[type] + 1
                    
    def __str__(self):
        if self.blnSplit == True:
            ret = 'Feature '+str(self.decisionAttribute)+'('+str(self.stat)+')'+'\n'
            for key in self.children.keys():
                ret = ret+'----key : '+str(key)+' '+str(self.children[key])
        else:
            ret = str(self.stat)+'\n'
        return ret
    
    def splitNode(self):
        self.blnSplit = True
        gains = self.calculateInformationGainPerFeatures()
        idxMaxGainFeature = -1
        maxGain = -999999999999999999.0
        for itr in range(len(gains)):
            if maxGain < gains[itr]:
                maxGain = gains[itr]
                idxMaxGainFeature = itr
                
        for value in Record.values:
            childRecords = []
            for record in self.records:
                if record.feature[idxMaxGainFeature] == value:
                    childRecords.append(record)
            self.children[value] = Node(childRecords)
        self.decisionAttribute = idxMaxGainFeature
        return self.children
    
    def calculateInformationGainPerFeatures(self):
        gains = []
        entropyClass = self.calculateClassEntropy()
        for itr in range(Record.numValues):
            entropyConditional = self.calculateConditionalEntropy(itr)
            gains.append(entropyClass-entropyConditional)
        return gains
    
    def calculateClassEntropy(self):
        entropy = 0.0
        for type in Record.types:
            cnt = 0.0
            for record in self.records:
                if record.party == type:
                    cnt = cnt + 1.0
            size = float(len(self.records))
            prob = float(cnt / size)
            entropy = entropy - prob * math.log(prob, 2)
        return entropy
    
    def calculateConditionalEntropy(self, idxFeature):
        entropy = 0.0
        for value in Record.values:
            for type in Record.types:    # 먼저 yes한 공화당원이라 생각하자.
                cntFeature = 0.0
                cntFeatureAndClass = 0.0
                for record in self.records:    # 해당 정치인 투표결과 record.
                    if record.feature[idxFeature] == value:    # 해당 정치인이 idx번째 법안에 대해 yes를 했다면,
                        cntFeature = cntFeature + 1
                        if record.party == type:    # 그리고 그 정치인이 공화당원이라면,
                            cntFeatureAndClass = cntFeatureAndClass + 1.0
                size = float(len(self.records))
                probFeature = cntFeature / size + 0.000001
                probFeatureAndClass = cntFeatureAndClass / size + 0.000001
                entropy = entropy + probFeatureAndClass * math.log(probFeature/probFeatureAndClass, 2)
        return entropy
    
if __name__ == "__main__":
    csvfile = open('house-votes-84.csv','rt')    
    reader = csv.reader(csvfile,delimiter=',')
    records = []

    for row in reader:
        record = Record(row)   
        print (record) 
        records.append(record)

    node = Node(records)
    print(node)
    node.splitNode()
    print(node)

# 연습

In [41]:
import csv
import math

In [5]:
import csv
import math

class Record:

    types = ['republican','democrat']
    values = ['y','n','?']
    numValues = 16

    def __init__(self,csvline):
        self.party = csvline[0]    # 정당 추출
        self.feature = []
        for itr in range(1,len(csvline)):
            self.feature.append(csvline[itr])    # 나머지 결과물 값을 feature에 저장.

    def __str__(self):
        ret = 'Classficiation : '+self.party+', Features : '+str(self.feature)    # 출력될 때 위에서 저장된 값들 표시.
        return ret
    
csvfile = open('house-votes-84.csv', 'rt')
reader = csv.reader(csvfile, delimiter=',')
records = []

for row in reader:
    record = Record(row)
    records.append(record)

stat = {}    
for type in Record.types:    # 공화당, 민주당 둘 중 하나
    stat[type] = 0    # 각각 정당의 type란에 0 저장.
    for record in records:
        if record.party == type:    # 해당 record 라인의 정당이 type과 일치한다면
            stat[type] = stat[type] + 1    # 해당 값에 1 더하기.  

# print(stat)


def calculateClassEntropy():
    entropy = 0.0    # 초기 엔트로피 값.
    for type in Record.types:    # 공화/민주
        cnt = 0.0
        for record in records:    # 밑에서 저장한 records값.
            if record.party == type:    #
                cnt = cnt + 1.0
        size = float(len(records))    # 전체 레코드 수.
        prob = float(cnt / size)    #
        entropy = entropy - prob * math.log(prob, 2)
    return entropy
    
# calculateClassEntropy()



0.5300898855559345
0.9623080486960709


0.9623080486960709

In [9]:
    def calculateConditionalEntropy(idxFeature):
        entropy = 0.0
        for value in Record.values:    # y, n, ? 세개.
            for type in Record.types:    # 공화/민주
                cntFeature = 0.0
                cntFeatureAndClass = 0.0
                for record in records:
                    if record.feature[idxFeature] == value:
                        cntFeature = cntFeature + 1
                        if record.party == type:
                            cntFeatureAndClass = cntFeatureAndClass + 1.0
                size = float(len(records))
                probFeature = cntFeature / size + 0.000001
                probFeatureAndClass = cntFeatureAndClass / size + 0.000001
                entropy = entropy + probFeatureAndClass * math.log(probFeature/probFeatureAndClass, 2)
        return entropy

In [11]:
for itr in range(16):    # 0부터 15 총 16개.
    entropyConditional = calculateConditionalEntropy(itr)   
    print(entropyConditional)

0.8362378942240004
0.9619493060825638
0.5299935852029543
0.22228413737143013
0.5398630143616605
0.8150771679569329
0.7646276771075199
0.6220862810623345
0.6517557586533733
0.9572280633109926
0.8550188844974302
0.5880608771647896
0.7345102487791235
0.6270300614352008
0.7419094152078063
0.860331965637684


In [22]:
h = 0
d = 2
n = -1
V = 5
cnt = 1
while True:
    h = h + d
    if h == V:
        break
    h = h + n
    cnt += 1
cnt

4

In [24]:
A, B, V = map(int, input().split())
h = 0
cnt = 1
while True:
    h = h + A
    if h == V:
        break
    h = h - B
    cnt += 1
print(cnt)

2 1 5


4

In [17]:
A, B, V = map(int, input().split())



1

In [40]:
A, B, V = map(int, input().split())
D = (V / (A-B))
if D > int(D):
    round(D)
print(D)

2 1 5
5.0


In [37]:
3//2

1

In [39]:
A, B, V = list(map(int, input().split()))
print((V - B - 1) // (A - B) + 1)

2 1 5
4
